In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def dm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="MSE", power = 2):
    # Routine for checking errors
    def error_check():
        rt = 0
        msg = ""
        # Check if h is an integer
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        # Check the range of h
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        # Check if lengths of actual values and predicted values are equal
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        # Check range of h
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)
        # Check if criterion supported
        if (crit != "MSE" and crit != "QLIKE" and crit != "MAD" and crit != "poly"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)
        # Check if every value of the input lists are numerical values
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")
        def compiled_regex(s):
            """ Returns True is string is a number. """
            if comp.match(s) is None:
                return s.isdigit()
            return True
        #for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
        #    is_actual_ok = compiled_regex(str(abs(actual)))
        #    is_pred1_ok = compiled_regex(str(abs(pred1)))
        #    is_pred2_ok = compiled_regex(str(abs(pred2)))
        #    if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):
        #        msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
        #        rt = -1
        #        return (rt,msg)
        return (rt,msg)

    # Error check
    error_code = error_check()
    # Raise error if cannot pass error check
    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return
    # Import libraries
    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np

    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst  = []

    # convert every value of the lists into real values
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()

    # Length of lists (as real numbers)
    T = float(len(actual_lst))

    # construct d according to crit
    if (crit == "MSE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append((actual - p1)**2)
            e2_lst.append((actual - p2)**2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAD"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "QLIKE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append((abs(actual)/abs(p1)-np.log(abs(actual)/abs(p1))-1))
            e2_lst.append((abs(actual)/abs(p2)-np.log(abs(actual)/abs(p2))-1))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "poly"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(((actual - p1))**(power))
            e2_lst.append(((actual - p2))**(power))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)

    # Mean of d
    mean_d = pd.Series(d_lst).mean()

    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)

    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')

    rt = dm_return(DM = DM_stat, p_value = p_value)

    return rt

In [ ]:
df=pd.read_excel("Statistical Tests - Overall.xlsx", index_col= "Date")

In [ ]:
df

,Stocks,Real Date,Actuals,NBEATSx,NBEATSx-2DWD,TFT,TFT-2DWD,HAR,GARCH-N,GARCH-T,GARCH-G
Date,,,,,,,,,,,
2017-04-20,AAPL,NaN,0.005300,0.004939,0.005636,0.005368,0.004715,0.006420,0.008295,0.007687,0.007945
2017-04-21,AAPL,NaN,0.005260,0.005127,0.005817,0.005425,0.004879,0.007370,0.008401,0.007768,0.008034
2017-04-24,AAPL,NaN,0.004482,0.005034,0.006010,0.005458,0.004987,0.007969,0.008503,0.007848,0.008122
2017-04-25,AAPL,NaN,0.004002,0.005118,0.005803,0.005478,0.005068,0.008348,0.008603,0.007927,0.008208
2017-04-26,AAPL,NaN,0.005834,0.005033,0.005944,0.005491,0.005135,0.008588,0.008700,0.008004,0.008292
...,...,...,...,...,...,...,...,...,...,...,...
2021-06-25,WMT,NaN,0.005518,0.008063,0.008370,0.008418,0.008178,0.009259,0.011986,0.012036,0.011955
2021-06-28,WMT,NaN,0.004666,0.008235,0.008633,0.008419,0.008182,0.009259,0.012029,0.012076,0.011995
2021-06-29,WMT,NaN,0.005565,0.008240,0.008637,0.008419,0.008185,0.009259,0.010013,0.010013,0.010013


In [ ]:
df.drop(columns=["Stocks", "Real Date"], inplace=True)

# DM tests with MSE

In [ ]:
dm_test(df["Actuals"], df["NBEATSx-2DWD"], df["NBEATSx"], h = 1, crit="MSE")

dm_return(DM=-60.69780431632514, p_value=0.0)

In [ ]:
dm_test(df["Actuals"], df["NBEATSx"], df["TFT"], h = 1, crit="MSE")

dm_return(DM=5.461938203398265, p_value=4.723429700843475e-08)

In [ ]:
dm_test(df["Actuals"], df["TFT-2DWD"], df["TFT"], h = 1, crit="MSE")

dm_return(DM=-6.7541134918226655, p_value=1.4467580827408761e-11)

# DM tests with MAE

In [ ]:
dm_test(df["Actuals"], df["NBEATSx-2DWD"], df["NBEATSx"], h = 1, crit="MAD")

dm_return(DM=-75.01532443324598, p_value=0.0)

In [ ]:
dm_test(df["Actuals"], df["NBEATSx"], df["TFT"], h = 1, crit="MAD")

dm_return(DM=13.678016852310643, p_value=1.5318869355143384e-42)

In [ ]:
dm_test(df["Actuals"], df["TFT-2DWD"], df["TFT"], h = 1, crit="MAD")

dm_return(DM=-12.161719655016153, p_value=5.321169475378462e-34)

# DM tests with QLIKE

In [ ]:
dm_test(df["Actuals"], df["NBEATSx-2DWD"], df["NBEATSx"], h = 1, crit="QLIKE")

dm_return(DM=-50.38884763900445, p_value=0.0)

In [ ]:
dm_test(df["Actuals"], df["NBEATSx"], df["TFT"], h = 1, crit="QLIKE")

dm_return(DM=33.55304654408259, p_value=3.924791730279786e-245)

In [ ]:
dm_test(df["Actuals"], df["TFT-2DWD"], df["TFT"], h = 1, crit="QLIKE")

dm_return(DM=12.64353088471913, p_value=1.3154312230115303e-36)

# MCS test with MSE

In [ ]:
!pip install arch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.4/916.4 kB 7.2 MB/s eta 0:00:00


In [ ]:
df1=df
df1["HAR"]=(df1["Actuals"]-df1["HAR"])**2
df1["NBEATSx"]=(df1["Actuals"]-df1["NBEATSx"])**2
df1["NBEATSx-2DWD"]=(df1["Actuals"]-df1["NBEATSx-2DWD"])**2
df1["TFT"]=(df1["Actuals"]-df1["TFT"])**2
df1["TFT-2DWD"]=(df1["Actuals"]-df1["TFT-2DWD"])**2
df1["GARCH-N"]=(df1["Actuals"]-df1["GARCH-N"])**2
df1["GARCH-T"]=(df1["Actuals"]-df1["GARCH-T"])**2
df1["GARCH-G"]=(df1["Actuals"]-df1["GARCH-G"])**2

In [ ]:
df1

,Actuals,NBEATSx,NBEATSx-2DWD,TFT,TFT-2DWD,HAR,GARCH-N,GARCH-T,GARCH-G
Date,,,,,,,,,
2017-04-20,0.005300,1.302371e-07,1.127569e-07,4.656807e-09,3.422468e-07,0.000001,0.000009,0.000006,0.000007
2017-04-21,0.005260,1.767798e-08,3.102662e-07,2.711521e-08,1.451472e-07,0.000004,0.000010,0.000006,0.000008
2017-04-24,0.004482,3.052172e-07,2.336653e-06,9.538242e-07,2.551777e-07,0.000012,0.000016,0.000011,0.000013
2017-04-25,0.004002,1.244938e-06,3.242085e-06,2.179284e-06,1.136695e-06,0.000019,0.000021,0.000015,0.000018
2017-04-26,0.005834,6.408535e-07,1.222065e-08,1.175248e-07,4.880003e-07,0.000008,0.000008,0.000005,0.000006
...,...,...,...,...,...,...,...,...,...
2021-06-25,0.005518,6.480306e-06,8.137773e-06,8.410555e-06,7.077551e-06,0.000014,0.000042,0.000042,0.000041
2021-06-28,0.004666,1.273909e-05,1.573902e-05,1.408333e-05,1.236323e-05,0.000021,0.000054,0.000055,0.000054
2021-06-29,0.005565,7.158498e-06,9.436023e-06,8.148792e-06,6.866676e-06,0.000014,0.000020,0.000020,0.000020


In [ ]:
from arch.bootstrap import MCS

In [ ]:
losses=df.drop(columns=["Actuals"])

In [ ]:
mcs = MCS(losses, size=0.01, method="R", block_size=10000)
mcs.compute()
print("MCS P-values")
print(mcs.pvalues)
print("Included")
included = mcs.included
print(included)
print("Excluded")
excluded = mcs.excluded
print(excluded)

MCS P-values
              Pvalue
Model name          
TFT-2DWD         0.0
NBEATSx          0.0
TFT              0.0
HAR              0.0
GARCH-G          0.0
GARCH-N          0.0
GARCH-T          0.0
NBEATSx-2DWD     1.0
Included
['NBEATSx-2DWD']
Excluded
['GARCH-G', 'GARCH-N', 'GARCH-T', 'HAR', 'NBEATSx', 'TFT', 'TFT-2DWD']


# MCS test with MAE

In [ ]:
df=pd.read_excel("Statistical Tests - Overall.xlsx", index_col= "Date")

In [ ]:
df1=df
df1["HAR"]=abs(df1["Actuals"]-df1["HAR"])
df1["NBEATSx"]=abs(df1["Actuals"]-df1["NBEATSx"])
df1["NBEATSx-2DWD"]=abs(df1["Actuals"]-df1["NBEATSx-2DWD"])
df1["TFT"]=abs(df1["Actuals"]-df1["TFT"])
df1["TFT-2DWD"]=abs(df1["Actuals"]-df1["TFT-2DWD"])
df1["GARCH-N"]=abs(df1["Actuals"]-df1["GARCH-N"])
df1["GARCH-T"]=abs(df1["Actuals"]-df1["GARCH-T"])
df1["GARCH-G"]=abs(df1["Actuals"]-df1["GARCH-G"])

In [ ]:
df1.head()

,Stocks,Real Date,Actuals,NBEATSx,NBEATSx-2DWD,TFT,TFT-2DWD,HAR,GARCH-N,GARCH-T,GARCH-G
Date,,,,,,,,,,,
2017-04-20,AAPL,NaN,0.005300,0.000361,0.000336,0.000068,0.000585,0.001121,0.002996,0.002387,0.002645
2017-04-21,AAPL,NaN,0.005260,0.000133,0.000557,0.000165,0.000381,0.002110,0.003141,0.002508,0.002774
2017-04-24,AAPL,NaN,0.004482,0.000552,0.001529,0.000977,0.000505,0.003488,0.004022,0.003366,0.003640
2017-04-25,AAPL,NaN,0.004002,0.001116,0.001801,0.001476,0.001066,0.004346,0.004601,0.003925,0.004206
2017-04-26,AAPL,NaN,0.005834,0.000801,0.000111,0.000343,0.000699,0.002754,0.002866,0.002170,0.002458


In [ ]:
losses=df.drop(columns=["Actuals", "Stocks","Real Date"])

In [ ]:
mcs = MCS(losses, size=0.01, method="R", block_size=10000)
mcs.compute()
print("MCS P-values")
print(mcs.pvalues)
print("Included")
included = mcs.included
print(included)
print("Excluded")
excluded = mcs.excluded
print(excluded)

MCS P-values
              Pvalue
Model name          
NBEATSx          0.0
TFT              0.0
TFT-2DWD         0.0
GARCH-T          0.0
GARCH-G          0.0
GARCH-N          0.0
HAR              0.0
NBEATSx-2DWD     1.0
Included
['NBEATSx-2DWD']
Excluded
['GARCH-G', 'GARCH-N', 'GARCH-T', 'HAR', 'NBEATSx', 'TFT', 'TFT-2DWD']


# MCS test with QLIKE

In [ ]:
df=pd.read_excel("Statistical Tests - Overall.xlsx", index_col= "Date")

In [ ]:
df1=df
df1["HAR"]=(df1["Actuals"]/df1["HAR"])-np.log(df1["Actuals"]/df1["HAR"])-1
df1["NBEATSx"]=(df1["Actuals"]/df1["NBEATSx"])-np.log(df1["Actuals"]/df1["NBEATSx"])-1
df1["NBEATSx-2DWD"]=(df1["Actuals"]/df1["NBEATSx-2DWD"])-np.log(df1["Actuals"]/df1["NBEATSx-2DWD"])-1
df1["TFT"]=(df1["Actuals"]/df1["TFT"])-np.log(df1["Actuals"]/df1["TFT"])-1
df1["TFT-2DWD"]=(df1["Actuals"]/df1["TFT-2DWD"])-np.log(df1["Actuals"]/df1["TFT-2DWD"])-1
df1["GARCH-N"]=(df1["Actuals"]/df1["GARCH-N"])-np.log(df1["Actuals"]/df1["GARCH-N"])-1
df1["GARCH-T"]=(df1["Actuals"]/df1["GARCH-T"])-np.log(df1["Actuals"]/df1["GARCH-T"])-1
df1["GARCH-G"]=(df1["Actuals"]/df1["GARCH-G"])-np.log(df1["Actuals"]/df1["GARCH-G"])-1

In [ ]:
df1.head()

,Stocks,Real Date,Actuals,NBEATSx,NBEATSx-2DWD,TFT,TFT-2DWD,HAR,GARCH-N,GARCH-T,GARCH-G
Date,,,,,,,,,,,
2017-04-20,AAPL,NaN,0.005300,0.002546,0.001849,0.000081,0.007115,0.017272,0.086911,0.061289,0.071936
2017-04-21,AAPL,NaN,0.005260,0.000331,0.004900,0.000470,0.002899,0.050974,0.094311,0.067010,0.078283
2017-04-24,AAPL,NaN,0.004482,0.006503,0.039145,0.018217,0.005506,0.137991,0.167512,0.131335,0.146389
2017-04-25,AAPL,NaN,0.004002,0.027906,0.061202,0.044516,0.025819,0.214643,0.230492,0.188313,0.205866
2017-04-26,AAPL,NaN,0.005834,0.011450,0.000175,0.001872,0.008491,0.065989,0.070215,0.045138,0.055159


In [ ]:
losses=df.drop(columns=["Actuals", "Stocks","Real Date"])

In [ ]:
mcs = MCS(losses, size=0.01, method="R", block_size=10000)
mcs.compute()
print("MCS P-values")
print(mcs.pvalues)
print("Included")
included = mcs.included
print(included)
print("Excluded")
excluded = mcs.excluded
print(excluded)

MCS P-values
              Pvalue
Model name          
NBEATSx          0.0
TFT-2DWD         0.0
TFT              0.0
GARCH-T          0.0
GARCH-G          0.0
GARCH-N          0.0
HAR              0.0
NBEATSx-2DWD     1.0
Included
['NBEATSx-2DWD']
Excluded
['GARCH-G', 'GARCH-N', 'GARCH-T', 'HAR', 'NBEATSx', 'TFT', 'TFT-2DWD']
